In [1]:
# Print out  all expressions
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" #default 'last_expr'
# Wider cells
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

C:\Users\EfiaA\AppData\Local\Temp\ipykernel_12308\1820498571.py:5: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
# Imports
import pickle
import pandas as pd
import numpy as np

#Import regular expressions, for data processing
import re

import gensim

In [3]:
#from helpers import path_pdf,path_pkl

In [5]:
df = pd.read_pickle('Pickledfilestexts_norefs_clean.pkl')

In [6]:
#Select only papers from TPT
#df = df[df['publisher'] == 'TPT']

#Select only papers from AJP
df = df[df['publisher'] == 'APT']


KeyError: 'publisher'

## Removing stopwords and stemming

Now, we can remove the stopwords and do the stemming, leaving us with a list of documents, each of which is essentially a tokenized list of words.

In [7]:
field='Content'

In [8]:
def sent_to_words(list_sentences):
    return [gensim.utils.simple_preprocess(str(sentence), deacc=True) for sentence in list_sentences]  #deacc=True removes accent marks from tokens (incl. punctuations)

def remove_stopwords(tokens):
    return [[word for word in gensim.utils.simple_preprocess(str(doc)) if word not in STOPWORDS ] for doc in tokens]

def get_wordnet_pos(word): #Provide a POS tag
    """Map POS tag to first character lemmatize() accepts"""
    tag = pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN) #return NOUN by default

def lemmatize_token(token):
    return nltk.stem.WordNetLemmatizer().lemmatize(token, get_wordnet_pos(token))

def lemmatize(token_list):
    '''Input example: ["he", "matches", "the", "profile"]'''
    return [lemmatize_token(token) for token in token_list]

In [9]:
#Tokenize documents
data_words = sent_to_words(df[field])

In [10]:
#Remove stopwords
from gensim.parsing.preprocessing import STOPWORDS
data_words_nostops = remove_stopwords(data_words)

In [11]:
%%time
#Lemmatize words
import nltk
from nltk.corpus import wordnet
from nltk import pos_tag
nltk.download('wordnet',quiet=True)
#nltk.download('punkt',quiet=True)   #required by word_tokenize method
nltk.download('averaged_perceptron_tagger',quiet=True) #required by pos_tag method
data_words_lemmatized = [lemmatize(token_list) for token_list in data_words_nostops] 

CPU times: total: 1min 47s
Wall time: 1min 49s


In [14]:
len(data_words_lemmatized)
#print(data_words_lemmatized[1])

178

In [15]:
phrases = gensim.models.phrases.Phrases(data_words_lemmatized, min_count=10) 
bigram = gensim.models.phrases.Phraser(phrases)
#bigram_mod = gensim.models.phrases.Phraser(bigram)
def make_bigrams(list_tokenized_docs): 
    '''Input example: [['He','matches','the','profile']]'''
    return [bigram[doc] for doc in list_tokenized_docs]

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_lemmatized)

In [16]:
with open('AJP_words_bigrams.pkl','wb') as output: pickle.dump(data_words_bigrams,output)